In [ ]:
#Transformer
import torch
from transformers import MarianMTModel, MarianTokenizer
class LanguageTranslator:
    def __init__(self, model_identifier: str = 'Helsinki-NLP/opus-mt-en-fr'):
        self.tokenizer = MarianTokenizer.from_pretrained(model_identifier)
        self.model = MarianMTModel.from_pretrained(model_identifier)

    def preprocess_text(self, text: str) -> str:
        return text.lower()

    def translate_text_to_french(self, text: str) -> str:
        preprocessed_text = self.preprocess_text(text)
        encoded_text = self.tokenizer(preprocessed_text, return_tensors="pt")
        generated_tokens = self.model.generate(**encoded_text)
        translated_text = self.tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
        return translated_text

translator = LanguageTranslator()
input_text = "The cat sat on the mat."
output_text = translator.translate_text_to_french(input_text)
print(f"Translated text: {output_text}")


In [ ]:
###ANN
import torch
import torch.nn as nn
import torch.nn.functional as F
# Create a Model Class that inherits nn.Module
class Model(nn.Module):
 
  # output (3 classes of iris flowers)
  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    super().__init__() # instantiate our nn.Module
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x
# Pick a manual seed for randomization
torch.manual_seed(41)
# Create an instance of model
model = Model()
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
url = 'https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'
my_df = pd.read_csv(url)
my_df
# Change last column from strings to integers
my_df['variety'] = my_df['variety'].replace('Setosa', 0.0)
my_df['variety'] = my_df['variety'].replace('Versicolor', 1.0)
my_df['variety'] = my_df['variety'].replace('Virginica', 2.0)
my_df
# Train Test Split/ Set X, y
X = my_df.drop('variety', axis=1)
y = my_df['variety']
X_np = my_df.drop('variety', axis=1).to_numpy()
y_np = my_df['variety'].to_numpy()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
epochs = 100
losses = []
for i in range(epochs):
    model.train()
    y_pred = model(X_train_tensor)
    
    loss = criterion(y_pred, y_train_tensor)
    
    losses.append(loss.item())
    
    if (i + 1) % 10 == 0:
        print(f'Epoch {i+1}, Loss: {loss.item()}')
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
import matplotlib.pyplot as plt

train_losses = []
val_losses = []

for i in range(epochs):
  y_pred = model(X_train_tensor)
  train_loss = criterion(y_pred, y_train_tensor)

  optimizer.zero_grad()
  train_loss.backward()
  optimizer.step()
  train_losses.append(train_loss.item())

  val_y_pred = model(X_test_tensor)
  val_loss = criterion(val_y_pred, y_test_tensor)
  val_losses.append(val_loss.item())
  if (i + 1) % 10 == 0:
    print(f'Epoch {i+1}, Training Loss: {train_loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

model.eval()

with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    loss = criterion(y_pred_tensor, y_test_tensor)
    print("Test Loss:", loss.item())

#classification

correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test_tensor):
    y_val = model.forward(data)

    if y_test[i] == 0:
      x = "Setosa"
    elif y_test[i] == 1:
      x = 'Versicolor'
    else:
      x = 'Virginica'


    # Will tell us what type of flower class our network thinks it is
    print(f'{i+1}.)  {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')

    # Correct or not
    if y_val.argmax().item() == y_test[i]:
      correct +=1

print(f'We got {correct} correct!')

new_iris = torch.tensor([4.7, 3.2, 1.3, 0.2])

In [ ]:
# KNN
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
data = pd.read_csv("diabetes.csv")
data.head()
zero_not_accepted = ['Glucose', 'BloodPressure',
                     'SkinThickness', 'BMI', 'Insulin']
for col in zero_not_accepted:
    data[col] = data[col].replace(0, np.NaN)
    mean = int(data[col].mean(skipna=True))
    data[col] = data[col].replace(np.NaN, mean)
x = data.iloc[:,0:8]
y = data.iloc[:,8]
sns.heatmap(data.corr())
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 0)
def euclidean_distance(instance1, instance2):
    return np.sqrt(np.sum((instance1 - instance2) ** 2))

def knn_classifier(K, train_data, train_labels, test_data):
    predicted_labels = []
    for test_point in test_data:
        distances = []
        for train_point, label in zip(train_data, train_labels):
            dist = euclidean_distance(test_point, train_point)
            distances.append((dist, label))
        distances = sorted(distances)[:K]
        labels = [label for dist, label in distances]
        predicted_labels.append(max(set(labels), key=labels.count))
    return predicted_labels

def evaluate_knn(K_values, train_data, test_data, train_labels, test_labels):
    for k in K_values:
        predicted_labels = knn_classifier(k, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, predicted_labels)
        print(f"Value of K = {k}, Accuracy = {accuracy}")
        print("Actual Values:", test_labels)
        print("Predicted Values:", predicted_labels)
        print()
K_values = [2, 4, 5, 7]
evaluate_knn(K_values, x_train.values, x_test.values, y_train, y_test)

# Plotting example (you can add more visualizations as needed)
if x_test.shape[1] >= 2:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=x_test.iloc[:, 0], y=x_test.iloc[:, 1], hue=y_test, palette="Set2")
    plt.title("Scatter plot of two features with actual labels")
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.legend(title="Label", loc="best")
    plt.show()
else:
    print("Insufficient features for plotting")


In [ ]:
### Genetic Algorithm
import random
# Define the problem parameters
secret_word = 'agenda'
word_length = len(secret_word)
population_size = 50
generations = 100
# Create the initial population
population = []
for i in range(population_size):
    individual = ''.join(random.choice('abcdefghijklmnopqrstuvwxyz') for j in range(word_length))
    population.append(individual)
# Define the fitness function
def fitness(individual):
    score = 0
    for i in range(word_length):
        if individual[i] == secret_word[i]:
            score += 1
    return score
# Define the selection function
def selection(population):
    total_fitness = sum(fitness(individual) for individual in population)

    probability_wheel = []
    for individual in population:
        probability = fitness(individual) / total_fitness
        probability_wheel.append(probability)

    parents = []
    for _ in range(2):
        random_value = random.random()
        current_sum = 0
        for i, probability in enumerate(probability_wheel):
            current_sum += probability
            if current_sum >= random_value:
                parents.append(population[i])
                break

    return parents
def crossover(parents):
    parent1, parent2 = parents

    crossover_points = sorted(random.sample(range(1, len(parent1) - 1), 2))

    child1 = parent1[:crossover_points[0]] + parent2[crossover_points[0]:crossover_points[1]] + parent1[crossover_points[1]:]
    child2 = parent2[:crossover_points[0]] + parent1[crossover_points[0]:crossover_points[1]] + parent2[crossover_points[1]:]

    return child1, child2
# Define the mutation function
def mutation(individual, probability):
    individual_list = list(individual)
    for i in range(word_length):
        if random.random() < probability:
            individual_list[i] = random.choice('abcdefghijklmnopqrstuvwxyz')

    return ''.join(individual_list)
# Run the genetic algorithm
for generation in range(generations):
    print(f"Generation {generation + 1}")
    # Selection
    new_population = []
    for i in range(population_size // 2):
        parents = selection(population)
        # Crossover
        child1, child2 = crossover(parents)
        # Mutation
        child1 = mutation(child1, 0.1)
        child2 = mutation(child2, 0.1)
        new_population.append(child1)
        new_population.append(child2)

    # Keep the best individual from the previous generation
    new_population.insert(0, max(population, key=fitness))
    population = new_population

    # Print the best individual
    best_individual = max(population, key=fitness)
    print(f"Best individual: {best_individual}")
    if best_individual == secret_word:
        print("Solution found!")
        break
    

In [ ]:
# CSP
def assign_packages(packages, cities, city_capacity, allowed_cities):

    def is_consistent(package, city):

        if city_capacity.get(city, 0) <= 0:        #Capacity Chacking

            return False
        
        
        if package in allowed_cities and city not in allowed_cities[package]:
        
            return False
        
        return True

    def backtrack_assignment(assignment):
        
        
        if len(assignment) == len(packages):

            return assignment
        
        
        package = None

        for p in packages:
        
            if p not in assignment:
        
                package = p
        
                break
        
        
        for city in cities:

            if is_consistent(package, city):

                
                assignment[package] = city

                city_capacity[city] -= 1
                
                
                result = backtrack_assignment(assignment)
                
               
                if result is not None:

                    return result
                
                
                assignment.pop(package)

                city_capacity[city] += 1
        
    
        return None

    
    assignment = {}

    optimal_solution = backtrack_assignment(assignment)  #Backtracking to check optimal solution
    
    
    solutions = [(package, optimal_solution[package]) for package in packages]
    
    return solutions


packages = [1, 2, 3, 4]
cities = ['A', 'B', 'C']
city_capacity = {'A': 2, 'B': 3, 'C': 1}
allowed_cities = {1: ['A', 'B'], 2: ['B', 'C'], 3: ['A', 'C'], 4: ['A']}


solutions = assign_packages(packages, cities, city_capacity.copy(), allowed_cities)


for package, city in solutions:
    print(f"Package {package} is assigned to City {city}")

In [ ]:
#A piori
import heapq

class MapGraph:
    def __init__(self, adjac_lis):
        self.adjac_lis = adjac_lis
 
    def get_neighbors(self, v):
        return self.adjac_lis[v]
 
    # This is a heuristic function which has equal values for all nodes
    def heuristic(self, n):
        H = {
            'Arad': 366,
            'Oradea': 380,
            'Timisoara': 329,
            'Zerind': 374,
            'Lugoj': 244,
            'Mehadia': 241, 
            'Drobeta': 242,
            'Craiova': 160,
            'Sibiu': 253,
            'Fagaras': 176,
            'Rimnicu Vilcea': 193,
            'Pitesti': 100,
            'Bucharest': 0
        }
        return H[n]

    def a_star_algorithm(self, start, goal):
        open_set = set([start])
        came_from = {}
        cost_so_far = {node: float('infinity') for node in self.adjac_lis}
        cost_so_far[start] = 0
        estimated_total_cost = {node: float('infinity') for node in self.adjac_lis}
        estimated_total_cost[start] = self.heuristic(start)

        while open_set:
            current = None
            current_estimated_cost = None
            for node in open_set:
                if current is None or estimated_total_cost[node] < current_estimated_cost:
                    current = node
                    current_estimated_cost = estimated_total_cost[node]

            if current == goal:
                path = []
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                path.append(start)
                path.reverse()
                print("Optimal Path:", path)
                print("Path Cost:", cost_so_far[goal])
                return path, cost_so_far[goal]

            open_set.remove(current)
            for neighbor, cost in self.get_neighbors(current):
                new_cost = cost_so_far[current] + cost
                if new_cost < cost_so_far[neighbor]:
                    came_from[neighbor] = current
                    cost_so_far[neighbor] = new_cost
                    estimated_total_cost[neighbor] = new_cost + self.heuristic(neighbor)
                    if neighbor not in open_set:
                        open_set.add(neighbor)

        print("No path found")
        return None, None

adjacency_list = {
    'Arad': [('Timisoara', 118), ('Zerind', 75), ('Sibiu', 140)],
    'Timisoara': [('Arad', 118), ('Lugoj', 111)],
    'Oradea': [('Sibiu', 151), ('Zerind', 71)],
    'Zerind': [('Oradea', 71), ('Arad', 75)],
    'Lugoj': [('Timisoara', 111), ('Mehadia', 70)],
    'Mehadia': [('Lugoj', 70), ('Drobeta', 75)],
    'Drobeta': [('Mehadia', 75), ('Craiova', 120)],
    'Craiova': [('Drobeta', 120), ('Rimnicu Vilcea', 146), ('Pitesti', 138)],
    'Sibiu': [('Arad', 140), ('Oradea', 151), ('Fagaras', 99), ('Rimnicu Vilcea', 80)],
    'Rimnicu Vilcea': [('Sibiu', 80), ('Pitesti', 97), ('Craiova', 146)],
    'Fagaras': [('Sibiu', 99), ('Bucharest', 211)],
    'Pitesti': [('Bucharest', 101), ('Craiova', 138), ('Rimnicu Vilcea', 97)],
    'Bucharest': [('Pitesti', 101), ('Fagaras', 211)],
}

graph = MapGraph(adjacency_list)
graph.a_star_algorithm('Arad', 'Bucharest')
